## Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN','NAME','STATUS', and 'SPECIAL_CONSIDERATIONS'.
application_df = application_df.drop(columns = ['EIN', 'NAME','STATUS','SPECIAL_CONSIDERATIONS'])

In [3]:
# Determine the number of unique values in each column.
application_df.nunique()

,0
APPLICATION_TYPE,17
AFFILIATION,6
CLASSIFICATION,71
USE_CASE,5
ORGANIZATION,4
INCOME_AMT,9
ASK_AMT,8747
IS_SUCCESSFUL,2


In [4]:
# Look at APPLICATION_TYPE value counts to identify and replace with "Other"
application_df['APPLICATION_TYPE'].value_counts()

,count
APPLICATION_TYPE,
T3,27037
T4,1542
T6,1216
T5,1173
T19,1065
T8,737
T7,725
T10,528
T9,156


In [5]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = application_df['APPLICATION_TYPE'].value_counts()[application_df['APPLICATION_TYPE'].value_counts() < 500].index.tolist()

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure replacement was successful
application_df['APPLICATION_TYPE'].value_counts()

,count
APPLICATION_TYPE,
T3,27037
T4,1542
T6,1216
T5,1173
T19,1065
T8,737
T7,725
T10,528
Other,276


In [6]:
# Look at CLASSIFICATION value counts to identify and replace with "Other"
application_df['CLASSIFICATION'].value_counts()

,count
CLASSIFICATION,
C1000,17326
C2000,6074
C1200,4837
C3000,1918
C2100,1883
...,...
C4120,1
C8210,1
C2561,1


In [7]:
# You may find it helpful to look at CLASSIFICATION value counts >1
application_df['CLASSIFICATION'].value_counts()[application_df['CLASSIFICATION'].value_counts() > 1]

,count
CLASSIFICATION,
C1000,17326
C2000,6074
C1200,4837
C3000,1918
C2100,1883
C7000,777
C1700,287
C4000,194
C5000,116


In [8]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = application_df['CLASSIFICATION'].value_counts()[application_df['CLASSIFICATION'].value_counts() < 1000].index.tolist()

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure replacement was successful
application_df['CLASSIFICATION'].value_counts()

,count
CLASSIFICATION,
C1000,17326
C2000,6074
C1200,4837
Other,2261
C3000,1918
C2100,1883


In [9]:
# Look at ASK_AMT value counts to identify and replace with "Other"
application_df['ASK_AMT'].value_counts()

,count
ASK_AMT,
5000,25398
10478,3
15583,3
63981,3
6725,3
...,...
5371754,1
30060,1
43091152,1


In [10]:
# Find the range of values in the column 'ASK_AMT'
ask_amt_upper_range = application_df['ASK_AMT'].max()
ask_amt_lower_range = application_df['ASK_AMT'].min()

#Print the range
print(f"The range of values in the 'ASK_AMT' column is from {ask_amt_lower_range} to {ask_amt_upper_range}.")

The range of values in the 'ASK_AMT' column is from 5000 to 8597806340.


In [11]:
# Define the bins and labels for the buckets
bins = [0, 10000, 50000, 100000, 500000, 1000000, float('inf')]
labels = ['0-10K', '10K-50K', '50K-100K', '100K-500K', '500K-1M', '1M+']

# Create a new column 'ASK_AMT_BUCKET' with the bucket labels
application_df['ASK_AMT_BUCKET'] = pd.cut(application_df['ASK_AMT'], bins=bins, labels=labels)

# Drop the original ASK_AMT column
application_df = application_df.drop(columns=['ASK_AMT'])

# Print the DataFrame with the new bucket column
application_df

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,INCOME_AMT,IS_SUCCESSFUL,ASK_AMT_BUCKET
0,T10,Independent,C1000,ProductDev,Association,0,1,0-10K
1,T3,Independent,C2000,Preservation,Co-operative,1-9999,1,100K-500K
2,T5,CompanySponsored,C3000,ProductDev,Association,0,0,0-10K
3,T3,CompanySponsored,C2000,Preservation,Trust,10000-24999,1,0-10K
4,T3,Independent,C1000,Heathcare,Trust,100000-499999,1,100K-500K
...,...,...,...,...,...,...,...,...
34294,T4,Independent,C1000,ProductDev,Association,0,0,0-10K
34295,T4,CompanySponsored,C3000,ProductDev,Association,0,0,0-10K
34296,T3,CompanySponsored,C2000,Preservation,Association,0,0,0-10K
34297,T5,Independent,C3000,ProductDev,Association,0,1,0-10K


In [12]:
# Look at ASK_AMT_BUCKET value counts
application_df['ASK_AMT_BUCKET'].value_counts()

,count
ASK_AMT_BUCKET,
0-10K,25947
10K-50K,2398
100K-500K,2304
1M+,1577
50K-100K,1423
500K-1M,650


In [13]:
application_df.nunique()

,0
APPLICATION_TYPE,9
AFFILIATION,6
CLASSIFICATION,6
USE_CASE,5
ORGANIZATION,4
INCOME_AMT,9
IS_SUCCESSFUL,2
ASK_AMT_BUCKET,6


In [14]:
# Convert categorical data to numeric with `pd.get_dummies`
categorical_df = pd.get_dummies(application_df)

In [15]:
# Split data into features (X) and target (y)
X = categorical_df.drop(columns='IS_SUCCESSFUL')
y = categorical_df['IS_SUCCESSFUL']

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [16]:
# Scale the data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [17]:
# Importing Dependencies
import pandas as pd
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import train_test_split, RandomizedSearchCV
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LeakyReLU, PReLU, ELU
from tensorflow.keras.callbacks import EarlyStopping

In [19]:
# Hyperparameter tuning for Random Forest
param_dist = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
}
rf = RandomForestClassifier(random_state=1)
random_search = RandomizedSearchCV(rf, param_distributions=param_dist, n_iter=10, cv=5, n_jobs=-1, verbose=2)
random_search.fit(X_train_selected, y_train)
print("Best parameters:", random_search.best_params_)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
Best parameters: {'n_estimators': 100, 'min_samples_split': 10, 'min_samples_leaf': 4, 'max_depth': 30}


In [24]:
# Create interaction features
poly = PolynomialFeatures(interaction_only=True, include_bias=False)
X_train_interactions = poly.fit_transform(X_train_scaled)
X_test_interactions = poly.transform(X_test_scaled)

# Combine interaction features with original features
interaction_columns = poly.get_feature_names_out(X.columns)
X_train_combined = pd.DataFrame(X_train_interactions, columns=interaction_columns)
X_test_combined = pd.DataFrame(X_test_interactions, columns=interaction_columns)

# Feature selection using Random Forest
selector = SelectFromModel(RandomForestClassifier(n_estimators= 100, min_samples_split = 10, min_samples_leaf= 4, max_depth= 30, random_state=1))
selector.fit(X_train_combined, y_train)
selected_features = X_train_combined.columns[selector.get_support()]
X_train_selected = X_train_combined[selected_features]
X_test_selected = X_test_combined[selected_features]


In [20]:
# Define the neural network model
nn = Sequential([

    # Input Layer
    Dense(128, input_dim=X_train_selected.shape[1], kernel_regularizer=tf.keras.regularizers.l2(0.01)),
    LeakyReLU(alpha=0.1),  # Activation: LeakyReLU to handle negative values gracefully
    Dropout(0.3),  # Dropout: Regularization to reduce overfitting

    # First Hidden Layer
    Dense(512, kernel_regularizer=tf.keras.regularizers.l2(0.01)),
    PReLU(),  # Activation: PReLU (Parametric ReLU) for learnable rectified linear units
    Dropout(0.3),  # Dropout: Adds regularization

    # Second Hidden Layer
    Dense(256, kernel_regularizer=tf.keras.regularizers.l2(0.01)),
    ELU(alpha=1.0),  # Activation: ELU (Exponential Linear Unit) for smoother outputs
    Dropout(0.3),  # Dropout: Regularization to avoid overfitting

    # Third Hidden Layer
    Dense(128, kernel_regularizer=tf.keras.regularizers.l2(0.01)),
    tf.keras.layers.ReLU(),  # Activation: ReLU (Rectified Linear Unit)
    Dropout(0.3),  # Dropout: Adds regularization

    # Fourth Hidden Layer
    Dense(64, kernel_regularizer=tf.keras.regularizers.l2(0.01)),
    tf.keras.layers.Activation('tanh'),  # Activation: Tanh for values between -1 and 1
    Dropout(0.3),  # Dropout: Adds regularization

    # Fifth Hidden Layer
    Dense(32, kernel_regularizer=tf.keras.regularizers.l2(0.01)),
    tf.keras.layers.Activation('sigmoid'),  # Activation: Sigmoid for values between 0 and 1
    Dropout(0.3),  # Dropout: Regularization

    # Output Layer
    Dense(1, activation='sigmoid')  # Activation: Sigmoid for binary classification output
])

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.10/dist-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


In [21]:
# Compile the model
nn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model with early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
history = nn.fit(X_train_selected, y_train, epochs=100, batch_size=32, validation_split=0.2, callbacks=[early_stopping])

Epoch 1/100
644/644 ━━━━━━━━━━━━━━━━━━━━ 14s 15ms/step - accuracy: 0.6265 - loss: 2.7561 - val_accuracy: 0.7314 - val_loss: 0.6665
Epoch 2/100
644/644 ━━━━━━━━━━━━━━━━━━━━ 6s 9ms/step - accuracy: 0.7030 - loss: 0.6907 - val_accuracy: 0.7259 - val_loss: 0.6660
Epoch 3/100
644/644 ━━━━━━━━━━━━━━━━━━━━ 13s 13ms/step - accuracy: 0.7073 - loss: 0.6855 - val_accuracy: 0.7333 - val_loss: 0.6579
Epoch 4/100
644/644 ━━━━━━━━━━━━━━━━━━━━ 12s 16ms/step - accuracy: 0.7006 - loss: 0.6855 - val_accuracy: 0.7318 - val_loss: 0.6545
Epoch 5/100
644/644 ━━━━━━━━━━━━━━━━━━━━ 9s 13ms/step - accuracy: 0.6996 - loss: 0.6855 - val_accuracy: 0.7287 - val_loss: 0.6515
Epoch 6/100
644/644 ━━━━━━━━━━━━━━━━━━━━ 11s 14ms/step - accuracy: 0.7073 - loss: 0.6804 - val_accuracy: 0.7242 - val_loss: 0.6712
Epoch 7/100
644/644 ━━━━━━━━━━━━━━━━━━━━ 9s 11ms/step - accuracy: 0.7095 - loss: 0.6760 - val_accuracy: 0.7306 - val_loss: 0.6472
Epoch 8/100
644/644 ━━━━━━━━━━━━━━━━━━━━ 9s 13ms/step - accuracy: 0.6949 - loss: 0.6837

In [22]:
# Evaluate the model
loss, accuracy = nn.evaluate(X_test_selected, y_test, verbose=0)
print(f"Model Loss: {loss:.4f}, Model Accuracy: {accuracy:.4f}")

Model Loss: 0.6246, Model Accuracy: 0.7217


In [23]:
# Export our model to HDF5 file
nn.save('AlphabetSoupCharity_Optimization.h5')